## Project - 4 : Treasure Hunters Inc.

# A game where the player in a map has to reach the treasure overcoming the monsters and blockers in between the path.
# Goal : Get the optimal path from source to destination.

This implementation differs from previous implementation in 2 ways..
1. Using Epsilon - Greedy method to generate the policy 
2. The rewards are different in this case.

## Steps in Project

### 1. Initialize transition,action and rewards tables
### 2. Build and Run the Reinforcement learning model on the map.
### 3. Build the QL table
### 4. Identify the optimal path from QLearning table

### Basic imports

In [1]:
import pandas as pd
import random
import numpy as np

### Initializing the 25 x 5 transitions table with possible moves.

### There are 25 possible positions on the game board, 
### we are going to define a possible position to move for each of the positions in each direction.
### For each position we have 4 possible moves, UP, DOWN, LEFT, RIGHT, so each position in game board, player will have 4 possible ways to move.
### every invalid move has a -1.
### we account for invalid move when the move is not possible, 
### for example if we are on first row, going up is an invalid move.


In [2]:
import numpy as np

def intializeTransitions():
    
    transitions = np.zeros((25,5))
    for i in range(25):    
        for j in range(4):
            
            # evaluation for moving UP
            # we always have 0-4 values in first row, so if i < 5 then we are on the top row, so going up
            # from top row is invalid.
            # from any other value, moving up is just subtracting 5 from current position.
            if i < 5:
                transitions[i][0] = -1
            else:
                transitions[i][0] = i - 5
                
            # evaluation for moving DOWN
            # we always have 20-24 values in last row, so if i > 20 then we are on the bottom most row, so going DOWN
            # from bottom row is invalid.
            # from any other value, moving DOwN is just adding 5 from current position.
            
            if i >= 20:
                transitions[i][1] = -1
            else:
                transitions[i][1] = i + 5
                
            # evaluation for moving LEFT
            # we always have values divisible by 5 in first column, so if i % 5 ==0 then we are on the first column, 
            # so going LEFT is not possible.
            # from any other value, moving LEFT is just adding -1 from current position.
            
            if i % 5 == 0:
                transitions[i][2] = -1
            else:
                transitions[i][2] = i - 1
            
            # evaluation for moving RIGHT
            # we always have values divisible by 5 in the (last column + 1), so if (i+1) % 5 ==0 then we are on the last column, 
            # so going RIGHT is not possible.
            # from any other value, moving RIGHT is just adding +1 from current position.
            
            # check if rightmost column, then 'right' action invalid
            if (i+1) % 5 == 0:
                transitions[i][3] = -1
            else:
                transitions[i][3] = i + 1
            
            # Lets store the current positions as well.
            transitions[i][4] = i
    
    transitions = transitions.astype(int)
    return transitions

## Rewards Policy

### Initializing the 25 x 5 rewards table with possible transitions from transitions table and the game board.

### The Policy : 
### There are 25 possible position on the game board, 
### we are going to define a rewards for each possible state in the grid.
### Defining the rewards policy..
### Whenever the player encounters a Monster, the reward is -20
### Whenever the player encounters a Blocker, the reward is -20
### Whenever the player encounters the Treasure, the reward is +100
### Whenever the Player encounters a * , the reward is +5 so that the Player can move. 
### Invalid moves from transition tables get a 0 reward.

In [3]:
def initializeRewards(gameBoard, transitions):
    
    rewards = np.zeros((25,5))
    
    for i in range(25):
        
        for j in range(5):
            
            possibleState = transitions[i][j]
            
            # if possible state is an invalid move = -1 from tansition table,
            # then set the reward for that state as 0.
            if possibleState == -1:
                rewards[i][j] = 0
                continue
            
            # if its a neutral move, then reward is 0
            if possibleState == i:
                rewards[i][j] = 0
                continue
            
            # if possible state encounter a Monster or a Blocker, set the reward to -50 or -30 respectively.
            gameBoardRow = int(possibleState / 5)
            gameBoardColumn = int(possibleState % 5)
            if gameBoard[gameBoardRow][gameBoardColumn] == 'M':
                rewards[i][j] = -10
            elif gameBoard[gameBoardRow][gameBoardColumn] == 'B':
                rewards[i][j] = -5
            elif gameBoard[gameBoardRow][gameBoardColumn] == 'T':
                rewards[i][j] = 10
            elif gameBoard[gameBoardRow][gameBoardColumn] == '*':
                rewards[i][j] = 1

    return rewards

### initializing an array of possible actions for all possible positions on the given game board.
#### Hardcoding these values as they are constant, we can programmatically write this as well.
0 - UP
1 - DOWN
2 - LEFT
3 - RIGHT
4 - NO MOVES
For example : the first value has 1,3,4 indicating that the player at that position can move DOWN or RIGHT.

In [4]:
def initializePossibleActions():
    possibleActions = np.array([[1, 3, 4],
                               [1, 2, 3, 4],
                               [1, 2, 3, 4],
                               [1, 2, 3, 4],
                               [1, 2, 4],
                               [0, 1, 3, 4],
                               [0, 1, 2, 3, 4],
                               [0, 1, 2, 3, 4],
                               [0, 1, 2, 3, 4],
                               [0, 1, 2, 4],
                               [0, 1, 3, 4],
                               [0, 1, 2, 3, 4],
                               [0, 1, 2, 3, 4],
                               [0, 1, 2, 3, 4],
                               [0, 1, 2, 4],
                               [0, 1, 3, 4],
                               [0, 1, 2, 3, 4],
                               [0, 1, 2, 3, 4],
                               [0, 1, 2, 3, 4],
                               [0, 1, 2, 4],
                               [0, 3, 4],
                               [0, 2, 3, 4],
                               [0, 2, 3, 4],
                               [0, 2, 3, 4],
                                [0, 2, 4]])
    return possibleActions

### Set up the game board with all required data.

#### Note that our game board is 5 X 5, so on total we will have 25 possibile positions on the board.


In [5]:
def setupGame(game):
    transitions = intializeTransitions();
    rewards = initializeRewards(game, transitions)
    return rewards, transitions

## Reinforced Learning model - Epsilon Greedy Method

Exploration vs Exploitation

Exploration allows an agent to improve its current knowledge about each action, hopefully leading to long-term benefit. Improving the accuracy of the estimated action-values, enables an agent to make more informed decisions in the future.

Exploitation on the other hand, chooses the greedy action to get the most reward by exploiting the agent’s current action-value estimates. But by being greedy with respect to action-value estimates, may not actually get the most reward and lead to sub-optimal behaviour.
When an agent explores, it gets more accurate estimates of action-values. And when it exploits, it might get more reward. It cannot, however, choose to do both simultaneously, which is also called the exploration-exploitation dilemma.

Epsilon-Greedy Action Selection
Epsilon-Greedy is a simple method to balance exploration and exploitation by choosing between exploration and exploitation randomly.
The epsilon-greedy, where epsilon refers to the probability of choosing to explore, exploits most of the time with a small chance of exploring.

** src : https://www.geeksforgeeks.org/epsilon-greedy-algorithm-in-reinforcement-learning/

In [6]:
import random

def reinforcedLearningEpsilonGreedyModel(gameBoard, transitions, rewards, learningRate, discountFactor, epochs):
    actionsAvailable = initializePossibleActions()
    qLearnings = np.zeros((25,5))

    for i in range(epochs):
    
        initialState = 0
        terminalState = 24
        currentState = initialState
        #Keep moving forward until the goal state is reached
        while currentState != terminalState:
            # random choice of action at every particular state.
            action = random.choice(actionsAvailable[currentState])

            #move to the next state based on  randomly chosen action and transitions.
            nextState = transitions[currentState][action]
            futureRewards = []

            #identify and Add all rewards for all future actions..
            for nextPossibleAction in actionsAvailable[nextState]:
                futureRewards.append(qLearnings[nextState][nextPossibleAction])

            ## Identify maximum Q learning value and apply the formula
            qPrevious = qLearnings[currentState][action] 
            qValueToUpdate = (1 - learningRate) *  qPrevious + learningRate * (rewards[currentState][action]  + discountFactor * max(futureRewards))
            
            ## Exploration vs Exploitation.. GREEDY EPSILON MODEL
            probability = np.random.random()
            if probability < 0.1: #(0.1 is epsilon here..)
                qValueToUpdate = np.random.choice(3) # exploitation use case..
            else:
                # exploration usecase..
                qValueToUpdate = (1 - learningRate) *  qPrevious + learningRate * (rewards[currentState][action]  + discountFactor * max(futureRewards))
            
            #Update the Q table with new reward value
            qLearnings[currentState][action] = qValueToUpdate

            # go to next state.
            currentState = nextState
    return qLearnings

### Initital design for the map to Treasure.

In [7]:
# P - Player.
# M - Monster
# B - Blocker
# T - Treasure
# Help the Player to reach treasure.

mapToTreasure = [['P','*','*','M','*'],
                 ['*','B','M','*','*'],
                 ['*','*','*','*','M'],
                 ['B','*','B','*','M'],
                 ['*','*','*','*','T']]

rewards, transitions = setupGame(mapToTreasure)
play = reinforcedLearningEpsilonGreedyModel(gameBoard=mapToTreasure,transitions=transitions, rewards=rewards, learningRate = 0.5, discountFactor = 0.8, epochs=200)

In [8]:
# converting the data into dataframe for manipulation purposes.
play = pd.DataFrame(play)

## Lets print the play move scores..
print("Play move scores..")
print(play)

print("Transitions..")
print(transitions)

Play move scores..
           0          1         2          3         4
0   0.000000   5.509124  0.000000   4.420045  4.450867
1   0.000000  -0.707360  4.410433   3.849323  3.214497
2   0.000000  -3.447153  4.306623  -2.948441  2.774445
3   0.000000   6.754018  3.638970   5.394073  0.000000
4   0.000000   6.058478 -3.773465   0.000000  4.573018
5   0.000000   5.341170  0.000000  -1.331227  4.234074
6   4.371993   6.070089  1.000000  -4.391573  2.000000
7   3.809735   6.613494 -0.680430   6.756511  5.397445
8   2.000000   7.197846 -4.601649   6.031058  0.000000
9   3.923391  -4.880137  6.547081   0.000000  5.098762
10  3.136468   0.084173  0.000000   5.300657  4.371718
11 -0.553384   6.880486  5.280176   6.094064  5.422702
12 -4.430608   1.419706  5.653162   6.453120  0.000000
13  6.731366   2.000000  6.388735  -4.911876  5.783783
14  6.095322  -2.000000  6.402867   0.000000  4.979724
15  5.163299   5.974866  0.000000   6.649916  5.195237
16  6.346048   7.447651  0.092730   1.399764  

### Get the route from the run.

### if at all there is going to be any visited node already then, pick the next highly possible move from all the moves.

In [9]:
optimalrouteToTreasure = []
currentstate = 0
previousState=0
nextHighestValue=0
state = 0

while state != 24: #(terminal state)
    row = play.iloc[state]
    print(state, "-->", end=" ")
    # get the index maximum value from the dataframe row
    action = row.idxmax(axis=1)
    #transitions of current state and the action will identify the next state
    state = transitions[state][action]

    ## trying to avoid previously visited nodes if any.
    if (state in optimalrouteToTreasure):
        nextHighest = 4 #(next highest in 5(0-4) values is 3, so setting this to 4 and subtracting below)
        while(state in optimalrouteToTreasure):
            nextHighest=nextHighest-1
            nextHighestValue = np.sort(play.iloc[previousState])[nextHighest] # get next highest value
            i=0
            for value in row.tolist():    
                if value == nextHighestValue:
                    state = transitions[previousState][i]
                    break
                i=i+1
                
    optimalrouteToTreasure.append(state)
    previousState = state    

0 --> 5 --> 10 --> 11 --> 16 --> 21 --> 22 --> 23 --> 

In [10]:
print("The optimal route for the Player to reach the treasure from source  is ", optimalrouteToTreasure)

The optimal route for the Player to reach the treasure from source  is  [5, 10, 11, 16, 21, 22, 23, 24]
